In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

import os
import sys 

CLASSES_MODULE_PATH = "../../"
WEIGHT_FILE_PATH = "./"
MODELS_FOLDER = CLASSES_MODULE_PATH + 'models'

# appending a path
sys.path.append(CLASSES_MODULE_PATH) #CHANGE THIS LINE



In [40]:
from examples.pytorch.lenet import LeNet5

N_CLASSES = 10

# Load the original model
model = LeNet5(N_CLASSES)
model.eval()

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (tanh1): Tanh()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (tanh2): Tanh()
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (tanh3): Tanh()
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (tanh4): Tanh()
  (linear2): Linear(in_features=84, out_features=10, bias=True)
)

In [41]:
transf = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

test_dataset = datasets.MNIST('data', train=False, transform=transf, download=True)
len(test_dataset)

10000

In [42]:
# Experiments with the layer vulernability factor
from examples.pytorch.lenet import LeNet5Simulator
from tqdm.notebook import tqdm
from collections import defaultdict


N_INJECTABLE_LAYERS = 1
N_IMAGES = 10000
N_SIM_PER_IMAGE = 1

fault_robust_runs_by_layer= defaultdict(int)
correct_classifications = defaultdict(int)

test_images = np.random.choice(len(test_dataset), size=N_IMAGES, replace=False).astype(int)
images_results = {}
layer_results = {}

prog_bar = tqdm(total=N_INJECTABLE_LAYERS * N_IMAGES * N_SIM_PER_IMAGE)

for image_idx in test_images:
    img, label = test_dataset[image_idx]
    img = img.unsqueeze(0)
    output_vanilla = model(img)[0]
    pred_vanilla = output_vanilla.argmax(dim=1).item()
    images_results[image_idx] = {
        "label": label,
        "golden_prediction": pred_vanilla,
    }

for injectable_layer_id in range(N_INJECTABLE_LAYERS):
    error_simuator_net = LeNet5Simulator(N_CLASSES, selected_layer=injectable_layer_id)
    error_simuator_net.eval()

    images = {}
    for image_idx in test_images:
        label = images_results[image_idx]["label"]
        pred_vanilla = images_results[image_idx]["golden_prediction"] 
        images[image_idx] = {
            "label": label,
            "golden_prediction": pred_vanilla,
            "results": defaultdict(int),
        }
        for run in range(N_SIM_PER_IMAGE):
            output_corr = error_simuator_net(img)[0]
            # print(f'The shape is {img.shape}')
            pred = output_corr.argmax(dim=1).item()
            # print(f" Pred vs Label => ({pred},{label})")
            images[image_idx]["results"][pred] += 1

            prog_bar.update(1)
    layer_results[injectable_layer_id] = {
        "images": images
    }




  0%|          | 0/10000 [00:00<?, ?it/s]

In [43]:
# import json
# with open("lvf_campaign.json", "w") as f:
#     json.dump(layer_results, f)

accuracy = sum(1 for v in images_results.values() if v["label"] == v["golden_prediction"]) / N_IMAGES
print(images_results)
print(f'{accuracy=}')

accuracy=0.0801
